# Deviations from Normality Python Assignment

## Provided Colab Code
(with some personal preference modifications)

In [15]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn-v0_8')

plt.rcParams['figure.figsize'] = [5, 3]
plt.rcParams['figure.dpi'] = 300
#warnings.simplefilter(action='ignore', category=FutureWarning)

In [16]:
import requests
import zipfile
import os
import time

url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/6_Portfolios_2x3_CSV.zip'
local_zip_file = '6_Portfolios_2x3_CSV.zip'
local_path = 'C:/Users/Brady/Dropbox/School Files/FIN 5322 - Investment Analysis/Python Notebooks/Data Files'

response = requests.get(url)
with open(local_zip_file, 'wb') as file:
    file.write(response.content)

with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
    zip_ref.extractall(local_path)
    
time.sleep(0.5)
os.remove(local_zip_file)

In [17]:
ports_csv = '6_Portfolios_2x3.CSV'
ports = pd.read_csv(ports_csv, skiprows=15)
ports

,Unnamed: 0,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,192607,1.0874,0.9081,-0.0695,5.7168,1.8971,2.0066
1,192608,0.7030,1.5075,5.3842,2.7154,2.7196,5.6796
2,192609,-2.9117,-0.1359,-0.4374,1.4287,0.0808,-0.7928
3,192610,-3.8196,-4.3572,-2.0112,-3.5898,-2.3377,-3.9998
4,192611,3.1806,3.6608,2.0944,3.1292,2.9242,3.1934
...,...,...,...,...,...,...,...
8694,202309,0.0851,0.0606,0.0823,0.0781,0.0956,0.0627
8695,202310,0.0838,0.0622,0.0872,0.0776,0.0943,0.0593
8696,202311,0.0765,0.0635,0.0878,0.0756,0.0926,0.0569
8697,202312,0.0771,0.0595,0.0867,0.0782,0.1013,0.0607


In [18]:
ports2 = ports.iloc[0:1169, :]
portfolios = ['date', 'SMALL_LoBM', 'ME1_BM2', 'SMALL_HiBM', 'BIG_LoBM', 'ME2_BM2', 'BIG_HiBM']

ports2.columns = portfolios
ports2

,date,SMALL_LoBM,ME1_BM2,SMALL_HiBM,BIG_LoBM,ME2_BM2,BIG_HiBM
0,192607,1.0874,0.9081,-0.0695,5.7168,1.8971,2.0066
1,192608,0.7030,1.5075,5.3842,2.7154,2.7196,5.6796
2,192609,-2.9117,-0.1359,-0.4374,1.4287,0.0808,-0.7928
3,192610,-3.8196,-4.3572,-2.0112,-3.5898,-2.3377,-3.9998
4,192611,3.1806,3.6608,2.0944,3.1292,2.9242,3.1934
...,...,...,...,...,...,...,...
1164,202307,5.1330,6.3109,9.2635,2.9021,4.5827,6.9958
1165,202308,-7.8268,-5.5493,-5.3284,-0.8988,-2.8246,-5.5092
1166,202309,-6.8099,-5.8368,-6.0807,-5.4008,-2.7077,-3.0848
1167,202310,-9.1193,-6.0789,-6.2047,-1.7960,-3.6544,-4.3317


##### Convert to datetime

In [19]:
ports2['date'] = pd.to_datetime(ports2['date'], format='%Y%m').dt.strftime("%Y-%m")
ports2.head(10)

C:\Users\Brady\AppData\Local\Temp\ipykernel_30112\1538304336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ports2['date'] = pd.to_datetime(ports2['date'], format='%Y%m').dt.strftime("%Y-%m")


,date,SMALL_LoBM,ME1_BM2,SMALL_HiBM,BIG_LoBM,ME2_BM2,BIG_HiBM
0,1926-07,1.0874,0.9081,-0.0695,5.7168,1.8971,2.0066
1,1926-08,0.7030,1.5075,5.3842,2.7154,2.7196,5.6796
2,1926-09,-2.9117,-0.1359,-0.4374,1.4287,0.0808,-0.7928
3,1926-10,-3.8196,-4.3572,-2.0112,-3.5898,-2.3377,-3.9998
4,1926-11,3.1806,3.6608,2.0944,3.1292,2.9242,3.1934
5,1926-12,2.6862,1.8411,3.2639,2.9678,2.6214,2.2990
6,1927-01,-0.7991,-0.3782,3.9316,-0.2621,0.0321,4.0885
7,1927-02,3.7129,6.3784,6.6192,5.0044,3.5850,7.9880
8,1927-03,-1.8324,-0.9889,-3.1676,1.3025,0.2527,-2.5908
9,1927-04,-1.6710,0.8741,3.4796,3.0510,-0.7745,-0.4809


In [20]:
ports2.set_index('date', drop=True, inplace=True)
ports2.head(10)

,SMALL_LoBM,ME1_BM2,SMALL_HiBM,BIG_LoBM,ME2_BM2,BIG_HiBM
date,,,,,,
1926-07,1.0874,0.9081,-0.0695,5.7168,1.8971,2.0066
1926-08,0.7030,1.5075,5.3842,2.7154,2.7196,5.6796
1926-09,-2.9117,-0.1359,-0.4374,1.4287,0.0808,-0.7928
1926-10,-3.8196,-4.3572,-2.0112,-3.5898,-2.3377,-3.9998
1926-11,3.1806,3.6608,2.0944,3.1292,2.9242,3.1934
1926-12,2.6862,1.8411,3.2639,2.9678,2.6214,2.2990
1927-01,-0.7991,-0.3782,3.9316,-0.2621,0.0321,4.0885
1927-02,3.7129,6.3784,6.6192,5.0044,3.5850,7.9880
1927-03,-1.8324,-0.9889,-3.1676,1.3025,0.2527,-2.5908


In [21]:
ports2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1169 entries, 1926-07 to 2023-11
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SMALL_LoBM  1169 non-null   object
 1   ME1_BM2     1169 non-null   object
 2   SMALL_HiBM  1169 non-null   object
 3   BIG_LoBM    1169 non-null   object
 4   ME2_BM2     1169 non-null   object
 5   BIG_HiBM    1169 non-null   object
dtypes: object(6)
memory usage: 63.9+ KB


In [22]:
ports2 = ports2.apply(pd.to_numeric, errors='coerce').div(100)
ports2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1169 entries, 1926-07 to 2023-11
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SMALL_LoBM  1169 non-null   float64
 1   ME1_BM2     1169 non-null   float64
 2   SMALL_HiBM  1169 non-null   float64
 3   BIG_LoBM    1169 non-null   float64
 4   ME2_BM2     1169 non-null   float64
 5   BIG_HiBM    1169 non-null   float64
dtypes: float64(6)
memory usage: 63.9+ KB


In [23]:
print(ports2)
ports2.describe()

         SMALL_LoBM   ME1_BM2  SMALL_HiBM  BIG_LoBM   ME2_BM2  BIG_HiBM
date                                                                   
1926-07    0.010874  0.009081   -0.000695  0.057168  0.018971  0.020066
1926-08    0.007030  0.015075    0.053842  0.027154  0.027196  0.056796
1926-09   -0.029117 -0.001359   -0.004374  0.014287  0.000808 -0.007928
1926-10   -0.038196 -0.043572   -0.020112 -0.035898 -0.023377 -0.039998
1926-11    0.031806  0.036608    0.020944  0.031292  0.029242  0.031934
...             ...       ...         ...       ...       ...       ...
2023-07    0.051330  0.063109    0.092635  0.029021  0.045827  0.069958
2023-08   -0.078268 -0.055493   -0.053284 -0.008988 -0.028246 -0.055092
2023-09   -0.068099 -0.058368   -0.060807 -0.054008 -0.027077 -0.030848
2023-10   -0.091193 -0.060789   -0.062047 -0.017960 -0.036544 -0.043317
2023-11    0.098575  0.090556    0.104811  0.095666  0.076559  0.122209

[1169 rows x 6 columns]


,SMALL_LoBM,ME1_BM2,SMALL_HiBM,BIG_LoBM,ME2_BM2,BIG_HiBM
count,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000
mean,0.009693,0.012286,0.014156,0.009391,0.009505,0.011809
std,0.074671,0.069635,0.081043,0.052998,0.056277,0.071320
min,-0.324829,-0.300521,-0.338737,-0.288663,-0.281507,-0.351131
25%,-0.030594,-0.021252,-0.020667,-0.017834,-0.016239,-0.020570
50%,0.011473,0.014994,0.015802,0.012499,0.012184,0.013829
75%,0.050015,0.046300,0.048259,0.039591,0.035492,0.044634
max,0.595730,0.621965,0.834957,0.337660,0.519421,0.677486


## Modified Code for Downloading Risk-Free Rates and Excess Market Data

#### Download file and clean file

##### Download file

In [24]:
import requests
import zipfile
import os
import time

url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip'
local_zip_file = 'risk_free_rates_and_ex_market_data_CSV.zip'
local_path = 'C:/Users/Brady/Dropbox/School Files/FIN 5322 - Investment Analysis/Python Notebooks/Data Files'

response = requests.get(url)
with open(local_zip_file, 'wb') as file:
    file.write(response.content)

with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
    zip_ref.extractall(local_path)
    
time.sleep(0.5)
os.remove(local_zip_file)

##### Clean up excess rows

In [25]:
market_data_csv = 'F-F_Research_Data_Factors.CSV'
market_data = pd.read_csv(market_data_csv, skiprows=3)
market_data

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31
...,...,...,...,...,...
1265,2020,23.66,13.18,-46.67,0.45
1266,2021,23.56,-3.89,25.49,0.04
1267,2022,-21.60,-6.95,25.81,1.43
1268,2023,21.68,-3.22,-13.57,4.95


In [26]:
market_data2 = market_data.iloc[0:1169, :]
cols = ['date', 'Mkt-RF', 'SMB', 'HML', 'RF']
market_data2.columns = cols
market_data2

,date,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31
...,...,...,...,...,...
1164,202307,3.21,2.08,4.11,0.45
1165,202308,-2.39,-3.16,-1.06,0.45
1166,202309,-5.24,-2.51,1.52,0.43
1167,202310,-3.19,-3.87,0.19,0.47


##### Convert to datetime and set Index

In [27]:
market_data2['date'] = pd.to_datetime(market_data2['date'], format='%Y%m').dt.strftime('%Y-%m')
market_data2.head(10)

C:\Users\Brady\AppData\Local\Temp\ipykernel_30112\1812011140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_data2['date'] = pd.to_datetime(market_data2['date'], format='%Y%m').dt.strftime('%Y-%m')


,date,Mkt-RF,SMB,HML,RF
0,1926-07,2.96,-2.56,-2.43,0.22
1,1926-08,2.64,-1.17,3.82,0.25
2,1926-09,0.36,-1.40,0.13,0.23
3,1926-10,-3.24,-0.09,0.70,0.32
4,1926-11,2.53,-0.10,-0.51,0.31
5,1926-12,2.62,-0.03,-0.05,0.28
6,1927-01,-0.06,-0.37,4.54,0.25
7,1927-02,4.18,0.04,2.94,0.26
8,1927-03,0.13,-1.65,-2.61,0.30
9,1927-04,0.46,0.30,0.81,0.25


In [28]:
market_data2.set_index('date', drop=True, inplace=True)
market_data2.head(10)

,Mkt-RF,SMB,HML,RF
date,,,,
1926-07,2.96,-2.56,-2.43,0.22
1926-08,2.64,-1.17,3.82,0.25
1926-09,0.36,-1.40,0.13,0.23
1926-10,-3.24,-0.09,0.70,0.32
1926-11,2.53,-0.10,-0.51,0.31
1926-12,2.62,-0.03,-0.05,0.28
1927-01,-0.06,-0.37,4.54,0.25
1927-02,4.18,0.04,2.94,0.26
1927-03,0.13,-1.65,-2.61,0.30


##### Confirm data is clean and ready

In [29]:
market_data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1169 entries, 1926-07 to 2023-11
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mkt-RF  1169 non-null   object
 1   SMB     1169 non-null   object
 2   HML     1169 non-null   object
 3   RF      1169 non-null   object
dtypes: object(4)
memory usage: 45.7+ KB


In [30]:
market_data2 = market_data2.apply(pd.to_numeric, errors='coerce').div(100)
market_data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1169 entries, 1926-07 to 2023-11
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  1169 non-null   float64
 1   SMB     1169 non-null   float64
 2   HML     1169 non-null   float64
 3   RF      1169 non-null   float64
dtypes: float64(4)
memory usage: 45.7+ KB


In [31]:
print(market_data2)
market_data2.describe()

         Mkt-RF     SMB     HML      RF
date                                   
1926-07  0.0296 -0.0256 -0.0243  0.0022
1926-08  0.0264 -0.0117  0.0382  0.0025
1926-09  0.0036 -0.0140  0.0013  0.0023
1926-10 -0.0324 -0.0009  0.0070  0.0032
1926-11  0.0253 -0.0010 -0.0051  0.0031
...         ...     ...     ...     ...
2023-07  0.0321  0.0208  0.0411  0.0045
2023-08 -0.0239 -0.0316 -0.0106  0.0045
2023-09 -0.0524 -0.0251  0.0152  0.0043
2023-10 -0.0319 -0.0387  0.0019  0.0047
2023-11  0.0884 -0.0002  0.0164  0.0044

[1169 rows x 4 columns]


,Mkt-RF,SMB,HML,RF
count,1169.000000,1169.00000,1169.000000,1169.000000
mean,0.006745,0.00181,0.003440,0.002675
std,0.053455,0.03169,0.035677,0.002508
min,-0.291300,-0.17200,-0.138700,-0.000600
25%,-0.020200,-0.01620,-0.014000,0.000300
50%,0.010600,0.00080,0.001300,0.002200
75%,0.036500,0.01750,0.017500,0.004200
max,0.388500,0.36560,0.356100,0.013500


## Merge and Clean Dataframes

##### Merge Dataframes

In [32]:
all_data = pd.merge(ports2, market_data2, left_index=True, right_index=True)

In [33]:
print(all_data)
all_data.describe()

         SMALL_LoBM   ME1_BM2  SMALL_HiBM  BIG_LoBM   ME2_BM2  BIG_HiBM  \
date                                                                      
1926-07    0.010874  0.009081   -0.000695  0.057168  0.018971  0.020066   
1926-08    0.007030  0.015075    0.053842  0.027154  0.027196  0.056796   
1926-09   -0.029117 -0.001359   -0.004374  0.014287  0.000808 -0.007928   
1926-10   -0.038196 -0.043572   -0.020112 -0.035898 -0.023377 -0.039998   
1926-11    0.031806  0.036608    0.020944  0.031292  0.029242  0.031934   
...             ...       ...         ...       ...       ...       ...   
2023-07    0.051330  0.063109    0.092635  0.029021  0.045827  0.069958   
2023-08   -0.078268 -0.055493   -0.053284 -0.008988 -0.028246 -0.055092   
2023-09   -0.068099 -0.058368   -0.060807 -0.054008 -0.027077 -0.030848   
2023-10   -0.091193 -0.060789   -0.062047 -0.017960 -0.036544 -0.043317   
2023-11    0.098575  0.090556    0.104811  0.095666  0.076559  0.122209   

         Mkt-RF     SMB 

,SMALL_LoBM,ME1_BM2,SMALL_HiBM,BIG_LoBM,ME2_BM2,BIG_HiBM,Mkt-RF,SMB,HML,RF
count,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000,1169.00000,1169.000000,1169.000000
mean,0.009693,0.012286,0.014156,0.009391,0.009505,0.011809,0.006745,0.00181,0.003440,0.002675
std,0.074671,0.069635,0.081043,0.052998,0.056277,0.071320,0.053455,0.03169,0.035677,0.002508
min,-0.324829,-0.300521,-0.338737,-0.288663,-0.281507,-0.351131,-0.291300,-0.17200,-0.138700,-0.000600
25%,-0.030594,-0.021252,-0.020667,-0.017834,-0.016239,-0.020570,-0.020200,-0.01620,-0.014000,0.000300
50%,0.011473,0.014994,0.015802,0.012499,0.012184,0.013829,0.010600,0.00080,0.001300,0.002200
75%,0.050015,0.046300,0.048259,0.039591,0.035492,0.044634,0.036500,0.01750,0.017500,0.004200
max,0.595730,0.621965,0.834957,0.337660,0.519421,0.677486,0.388500,0.36560,0.356100,0.013500


##### Clean Dataframes

In [34]:
all_data.rename(columns={'SMALL_LoBM': 'Small/Growth',
                        'SMALL_HiBM': 'Small/Value',
                        'BIG_LoBM': 'Big/Growth',
                        'BIG_HiBM': 'Big/Value'}, inplace=True)
all_data.drop(columns=['ME1_BM2', 'ME2_BM2', 'SMB', 'HML'], inplace=True)

In [35]:
portfolio_columns = ['Small/Growth', 'Small/Value', 'Big/Growth', 'Big/Value']
rf_col = 'RF'

for column in portfolio_columns:
    all_data[column] = all_data[column] - all_data['RF']

In [36]:
print(all_data)
all_data.describe()

         Small/Growth  Small/Value  Big/Growth  Big/Value  Mkt-RF      RF
date                                                                     
1926-07      0.008674    -0.002895    0.054968   0.017866  0.0296  0.0022
1926-08      0.004530     0.051342    0.024654   0.054296  0.0264  0.0025
1926-09     -0.031417    -0.006674    0.011987  -0.010228  0.0036  0.0023
1926-10     -0.041396    -0.023312   -0.039098  -0.043198 -0.0324  0.0032
1926-11      0.028706     0.017844    0.028192   0.028834  0.0253  0.0031
...               ...          ...         ...        ...     ...     ...
2023-07      0.046830     0.088135    0.024521   0.065458  0.0321  0.0045
2023-08     -0.082768    -0.057784   -0.013488  -0.059592 -0.0239  0.0045
2023-09     -0.072399    -0.065107   -0.058308  -0.035148 -0.0524  0.0043
2023-10     -0.095893    -0.066747   -0.022660  -0.048017 -0.0319  0.0047
2023-11      0.094175     0.100411    0.091266   0.117809  0.0884  0.0044

[1169 rows x 6 columns]


,Small/Growth,Small/Value,Big/Growth,Big/Value,Mkt-RF,RF
count,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000,1169.000000
mean,0.007017,0.011481,0.006716,0.009133,0.006745,0.002675
std,0.074827,0.081153,0.053112,0.071393,0.053455,0.002508
min,-0.330829,-0.339037,-0.288963,-0.351431,-0.291300,-0.000600
25%,-0.033779,-0.023623,-0.020407,-0.022770,-0.020200,0.000300
50%,0.009372,0.012826,0.009811,0.010895,0.010600,0.002200
75%,0.048098,0.046060,0.036634,0.042301,0.036500,0.004200
max,0.595330,0.834657,0.336660,0.677186,0.388500,0.013500


## Build the Final Table

##### Define some functions for formula replication

In [37]:
def sharpe_ratio(excess_return, standard_deviation):
    sharpe_ratio = excess_return / standard_deviation
    return sharpe_ratio

##### Calculate Table Values for Sec. A - 1927-2021

###### Select Sec. A Data

In [38]:
# Select the date range for the first half of the table

all_data_27_21 = all_data.loc[(all_data.index >= '1927-01') & (all_data.index <= '2021-12')]
print(all_data_27_21)
all_data_27_21.describe()

         Small/Growth  Small/Value  Big/Growth  Big/Value  Mkt-RF      RF
date                                                                     
1927-01     -0.010491     0.036816   -0.005121   0.038385 -0.0006  0.0025
1927-02      0.034529     0.063592    0.047444   0.077280  0.0418  0.0026
1927-03     -0.021324    -0.034676    0.010025  -0.028908  0.0013  0.0030
1927-04     -0.019210     0.032296    0.028010  -0.007309  0.0046  0.0025
1927-05      0.051766     0.130256    0.051076   0.067144  0.0544  0.0030
...               ...          ...         ...        ...     ...     ...
2021-08      0.024939     0.027340    0.035727   0.030371  0.0291  0.0000
2021-09     -0.044684     0.005032   -0.054490  -0.002526 -0.0437  0.0000
2021-10      0.029734     0.037682    0.080178   0.062388  0.0665  0.0000
2021-11     -0.070132    -0.038005   -0.003202  -0.044293 -0.0155  0.0000
2021-12     -0.000385     0.043987    0.025878   0.046721  0.0310  0.0001

[1140 rows x 6 columns]


,Small/Growth,Small/Value,Big/Growth,Big/Value,Mkt-RF,RF
count,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000
mean,0.007447,0.011787,0.006819,0.009254,0.006900,0.002678
std,0.074933,0.081528,0.052987,0.071605,0.053468,0.002528
min,-0.330829,-0.339037,-0.288963,-0.351431,-0.291300,-0.000600
25%,-0.033100,-0.023333,-0.019848,-0.022663,-0.019700,0.000300
50%,0.009844,0.013047,0.009723,0.010868,0.010600,0.002200
75%,0.048752,0.046087,0.036532,0.041476,0.036500,0.004200
max,0.595330,0.834657,0.336660,0.677186,0.388500,0.013500


###### Calculate Stats

In [39]:
import numpy as np
import scipy.stats as stats

# Define the columns that stats will be calculated for

portfolio_cols = ['Mkt-RF', 'Big/Growth', 'Big/Value', 'Small/Growth', 'Small/Value']

# Initialize dictionaries to collect the statistics

portfolio_mean_27 = {}
portfolio_std_27 = {}
portfolio_sharpe_27 = {}
portfolio_LPSD_27 = {}
portfolio_skew_27 = {}
portfolio_kurt_27 = {}
portfolio_var_month_27 = {}
portfolio_var_normal_27 = {}
portfolio_3std_27 = {}
portfolio_shortfall_27 = {}


# Begin the loop to calculate all the stats from the BKM table

for col in portfolio_cols:
    # Calcualte the mean for all columns
    mean = all_data_27_21[col].mean()
    annual_mean = (1 + mean)**12 - 1
    portfolio_mean_27[col] = annual_mean

    # Calculate STD for all columns
    std_dev = all_data_27_21[col].std() * np.sqrt(12)
    portfolio_std_27[col] = std_dev

    # Calculate sharpe ratio for all columns
    sharpe = sharpe_ratio(annual_mean, std_dev)
    portfolio_sharpe_27[col] = sharpe

    # Calculate the LPSD for all columns
    below_mean = all_data_27_21[col][all_data_27_21[col] < mean]
    portfolio_LPSD_27[col] = below_mean.std() * np.sqrt(12)

    # Calculate skew for all columns
    skew = all_data_27_21[col].skew()
    portfolio_skew_27[col] = skew

    # Calculate kurtosis for all columns
    kurt = all_data_27_21[col].kurtosis()
    portfolio_kurt_27[col] = kurt

    # Calculate VaR 1% actual monthly returns
    var_month = np.percentile(all_data_27_21[col], 1)
    portfolio_var_month_27[col] = var_month

    # Calculate VaR 1% of normal distribution
    var_normal = stats.norm.ppf(0.01, mean, std_dev)
    portfolio_var_normal_27[col] = var_normal

    # Calculate the % of monthly returns 3 STDs below the mean
    three_std_below = mean - 3 * std_dev
    count_below = (all_data_27_21[col] < three_std_below).sum()
    total_entries = len(all_data_27_21[col])
    pct_3std = (count_below / total_entries) * 100
    portfolio_3std_27[col] = pct_3std

    # Calculate expected shortfall
    under_var = all_data_27_21[col][all_data_27_21[col] < var_month]
    expected_sf = under_var.mean()
    portfolio_shortfall_27[col] = expected_sf


# Display stats for verification

print(f"Mean : {portfolio_mean_27}")
print(f"Std: {portfolio_std_27}")
print(f"Sharpe Ratio: {portfolio_sharpe_27}")
print(f"Lower Partial SD: {portfolio_LPSD_27}")
print(f"Skew: {portfolio_skew_27}")
print(f"Kurtosis: {portfolio_kurt_27}")
print(f"VaR 1% Monthly: {portfolio_var_month_27}")
print(f"VaR 1% Normal: {portfolio_var_normal_27}")
print(f"% 3 Std's Below the Mean: {portfolio_3std_27}")
print(f"Expected Shortfall: {portfolio_shortfall_27}")

Mean : {'Mkt-RF': 0.0860202078607748, 'Big/Growth': 0.08496970782437785, 'Big/Value': 0.11687999720090048, 'Small/Growth': 0.09311546128122972, 'Small/Value': 0.15098556529704377}
Std: {'Mkt-RF': 0.18521765171274582, 'Big/Growth': 0.18355089579480915, 'Big/Value': 0.248045908555481, 'Small/Growth': 0.25957516359459587, 'Small/Value': 0.2824199698752817}
Sharpe Ratio: {'Mkt-RF': 0.46442769933280226, 'Big/Growth': 0.462921782301543, 'Big/Value': 0.471203084467558, 'Small/Growth': 0.35872253720952024, 'Small/Value': 0.5346136300620664}
Lower Partial SD: {'Mkt-RF': 0.13799338458302718, 'Big/Growth': 0.1353982392782604, 'Big/Value': 0.171449819677123, 'Small/Growth': 0.17792671021268422, 'Small/Value': 0.18858303576859978}
Skew: {'Mkt-RF': 0.16512921942168618, 'Big/Growth': -0.1148348553093589, 'Big/Value': 1.5056043256107547, 'Small/Growth': 0.5823644468138152, 'Small/Value': 2.0637947425386995}
Kurtosis: {'Mkt-RF': 7.613176786604974, 'Big/Growth': 5.464968188197446, 'Big/Value': 17.605147

##### Calculate Table Values for Sec. B - 1252-2021

###### Select Sec. B Data

In [40]:
# Select the date range for the second half of the table

all_data_52_21 = all_data.loc[(all_data.index >= '1952-01') & (all_data.index <= '2021-12')]
print(all_data_52_21)
all_data_52_21.describe()

         Small/Growth  Small/Value  Big/Growth  Big/Value  Mkt-RF      RF
date                                                                     
1952-01      0.015299     0.012797    0.002724   0.036217  0.0145  0.0015
1952-02     -0.021638    -0.020856   -0.022960  -0.035960 -0.0262  0.0012
1952-03      0.011454     0.032227    0.042610   0.065170  0.0444  0.0011
1952-04     -0.049013    -0.042323   -0.047720  -0.056371 -0.0497  0.0012
1952-05      0.021533     0.025753    0.036603   0.033277  0.0320  0.0013
...               ...          ...         ...        ...     ...     ...
2021-08      0.024939     0.027340    0.035727   0.030371  0.0291  0.0000
2021-09     -0.044684     0.005032   -0.054490  -0.002526 -0.0437  0.0000
2021-10      0.029734     0.037682    0.080178   0.062388  0.0665  0.0000
2021-11     -0.070132    -0.038005   -0.003202  -0.044293 -0.0155  0.0000
2021-12     -0.000385     0.043987    0.025878   0.046721  0.0310  0.0001

[840 rows x 6 columns]


,Small/Growth,Small/Value,Big/Growth,Big/Value,Mkt-RF,RF
count,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000
mean,0.006368,0.010244,0.006534,0.008143,0.006511,0.003355
std,0.064594,0.054950,0.044764,0.049290,0.043166,0.002557
min,-0.330829,-0.284467,-0.238158,-0.273567,-0.232400,0.000000
25%,-0.032068,-0.018991,-0.019355,-0.020163,-0.018925,0.001400
50%,0.009713,0.013047,0.009611,0.010655,0.010300,0.003100
75%,0.042828,0.042299,0.035305,0.038290,0.034000,0.004700
max,0.276583,0.301145,0.207335,0.206971,0.161000,0.013500


In [41]:
import numpy as np
import scipy.stats as stats

portfolio_cols = ['Mkt-RF', 'Big/Growth', 'Big/Value', 'Small/Growth', 'Small/Value']

portfolio_mean_52 = {}
portfolio_std_52 = {}
portfolio_sharpe_52 = {}
portfolio_LPSD_52 = {}
portfolio_skew_52 = {}
portfolio_kurt_52 = {}
portfolio_var_month_52 = {}
portfolio_var_normal_52 = {}
portfolio_3std_52 = {}
portfolio_shortfall_52 = {}

for col in portfolio_cols:
    # Calcualte the mean for all columns
    mean = all_data_52_21[col].mean()
    annul_mean = (1 + mean)**12 - 1
    portfolio_mean_52[col] = annual_mean

    # Calculate STD for all columns
    std_dev = all_data_52_21[col].std() * np.sqrt(12)
    portfolio_std_52[col] = std_dev

    # Calculate sharpe ratio for all columns
    sharpe = sharpe_ratio(mean, std_dev)
    portfolio_sharpe_52[col] = sharpe

    # Calculate the LPSD for all columns
    below_mean = all_data_52_21[col][all_data_52_21[col] < mean]
    portfolio_LPSD_52[col] = below_mean.std() * np.sqrt(12)

    # Calculate skew for all columns
    skew = all_data_52_21[col].skew()
    portfolio_skew_52[col] = skew

    # Calculate kurtosis for all columns
    kurt = all_data_52_21[col].kurtosis()
    portfolio_kurt_52[col] = kurt

    # Calculate VaR 1% actual monthly returns
    var_month = np.percentile(all_data_52_21[col], 1)
    portfolio_var_month_52[col] = var_month

    # Calculate VaR 1% of normal distribution
    var_normal = stats.norm.ppf(0.01, mean, std_dev)
    portfolio_var_normal_52[col] = var_normal

    # Calculate the % of monthly returns 3 STDs below the mean
    three_std_below = mean - 3 * std_dev
    count_below = (all_data_52_21[col] < three_std_below).sum()
    total_entries = len(all_data_52_21[col])
    pct_3std = (count_below / total_entries) * 100
    portfolio_3std_52[col] = pct_3std

    # Calculate expected shortfall
    under_var = all_data_52_21[col][all_data_52_21[col] < var_month]
    expected_sf = under_var.mean()
    portfolio_shortfall_52[col] = expected_sf

print(f"Mean : {portfolio_mean_52}")
print(f"Std: {portfolio_std_52}")
print(f"Sharpe Ratio: {portfolio_sharpe_52}")
print(f"Lower Partial SD: {portfolio_LPSD_52}")
print(f"Skew: {portfolio_skew_52}")
print(f"Kurtosis: {portfolio_kurt_52}")
print(f"VaR 1% Monthly: {portfolio_var_month_52}")
print(f"VaR 1% Normal: {portfolio_var_normal_52}")
print(f"% 3 Std's Below the Mean: {portfolio_3std_52}")
print(f"Expected Shortfall: {portfolio_shortfall_52}")

Mean : {'Mkt-RF': 0.15098556529704377, 'Big/Growth': 0.15098556529704377, 'Big/Value': 0.15098556529704377, 'Small/Growth': 0.15098556529704377, 'Small/Value': 0.15098556529704377}
Std: {'Mkt-RF': 0.1495311157196511, 'Big/Growth': 0.1550669129509577, 'Big/Value': 0.17074675139980686, 'Small/Growth': 0.22375946853276546, 'Small/Value': 0.19035341204786832}
Sharpe Ratio: {'Mkt-RF': 0.043540866089175165, 'Big/Growth': 0.04213409964491339, 'Big/Value': 0.04768853889449347, 'Small/Growth': 0.028459460738267363, 'Small/Value': 0.05381403156163828}
Lower Partial SD: {'Mkt-RF': 0.10989686120310085, 'Big/Growth': 0.10892813622252365, 'Big/Value': 0.12483455132652661, 'Small/Growth': 0.16143612958508394, 'Small/Value': 0.14416173809835287}
Skew: {'Mkt-RF': -0.5175459799435123, 'Big/Growth': -0.35163736058773015, 'Big/Value': -0.45565945796810386, 'Small/Growth': -0.3380071263809359, 'Small/Value': -0.4420605927828116}
Kurtosis: {'Mkt-RF': 1.9020459134204697, 'Big/Growth': 1.7479620432580503, 'Bi

In [42]:
df_27_21 = pd.DataFrame({
    'Mean': portfolio_mean_27,
    'Std Dev': portfolio_std_27,
    'Sharpe Ratio': portfolio_sharpe_27,
    'Lower Partial SD': portfolio_LPSD_27,
    'Skew': portfolio_skew_27,
    'Kurtosis': portfolio_kurt_27,
    'Var 1% Monthly': portfolio_var_month_27,
    'Var 1% Normal': portfolio_var_normal_27,
    "% 3 Std's Below the Mean": portfolio_3std_27,
    'Expected Shortfall': portfolio_shortfall_27,
}).T

df_52_21 = pd.DataFrame({
    'Mean': portfolio_mean_52,
    'Std Dev': portfolio_std_52,
    'Sharpe Ratio': portfolio_sharpe_52,
    'Lower Partial SD': portfolio_LPSD_52,
    'Skew': portfolio_skew_52,
    'Kurtosis': portfolio_kurt_52,
    'Var 1% Monthly': portfolio_var_month_52,
    'Var 1% Normal': portfolio_var_normal_52,
    "% 3 Std's Below the Mean": portfolio_3std_52,
    'Expected Shortfall': portfolio_shortfall_52,
}).T

combined_df = pd.concat([df_27_21, df_52_21], axis=1, keys=['1927-2021', '1952-2021'])

combined_df

1927-2021                                     \
                            Mkt-RF Big/Growth  Big/Value Small/Growth   
Mean                      0.086020   0.084970   0.116880     0.093115   
Std Dev                   0.185218   0.183551   0.248046     0.259575   
Sharpe Ratio              0.464428   0.462922   0.471203     0.358723   
Lower Partial SD          0.137993   0.135398   0.171450     0.177927   
Skew                      0.165129  -0.114835   1.505604     0.582364   
Kurtosis                  7.613177   5.464968  17.605148     7.055643   
Var 1% Monthly           -0.135788  -0.143843  -0.198874    -0.201022   
Var 1% Normal            -0.423980  -0.420184  -0.567787    -0.596415   
% 3 Std's Below the Mean  0.000000   0.000000   0.000000     0.000000   
Expected Shortfall       -0.195967  -0.198557  -0.244938    -0.246580   

                                     1952-2021                       \
                         Small/Value    Mkt-RF Big/Growth Big/Value   
Mean                        0.150986  0.150986   0.150986  0.150986   
Std Dev                     0.282420  0.149531   0.155067  0.170747   
Sharpe Ratio                0.534614  0.043541   0.042134  0.047689   
Lower Partial SD            0.188583  0.109897   0.108928  0.124835   
Skew                        2.063795 -0.517546  -0.351637 -0.455659   
Kurtosis                   21.554364  1.902046   1.747962  2.996623   
Var 1% Monthly             -0.207701 -0.108908  -0.108398 -0.124753   
Var 1% Normal              -0.645220 -0.341351  -0.354206 -0.389074   
% 3 Std's Below the Mean    0.000000  0.000000   0.000000  0.000000   
Expected Shortfall         -0.259843 -0.144744  -0.139658 -0.180374   

                                                   
                         Small/Growth Small/Value  
Mean                         0.150986    0.150986  
Std Dev                      0.223759    0.190353  
Sharpe Ratio                 0.028459    0.053814  
Lower Partial SD             0.161436    0.144162  
Skew                        -0.338007   -0.442061  
Kurtosis                     2.156983    3.814618  
Var 1% Monthly              -0.171956   -0.153147  
Var 1% Normal               -0.514174   -0.432585  
% 3 Std's Below the Mean     0.000000    0.000000  
Expected Shortfall          -0.227520   -0.203190

#### Export table to png

In [44]:
import dataframe_image as dfi

dfi.export(combined_df, 'portfolio_stats.png')